# 基础因子加alpha191回测结果

添加费后曲线

In [1]:
%matplotlib inline
import sys
sys.path.append('../')
sys.path.append('../../')
sys.path.append('../../../')
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

from PyFin.api import *
from alphamind.api import *
from conf.models import *
from conf.config import*
from alphamind.execution.naiveexecutor import NaiveExecutor
from stacking import factor_store, feature_list

data_source = alpha_db
engine = SqlEngine(data_source)

In [2]:
universe = Universe('zz500')
freq = '5b'
benchmark_code = 905
start_date = '2018-01-01'  # 训练集的起始时间
back_start_date = '2018-10-01'  # 回测的起始时间
end_date = '2019-10-01'
ref_dates = makeSchedule(start_date, end_date, freq, 'china.sse')
back_ref_dates = makeSchedule(back_start_date, end_date, freq, 'china.sse')
horizon = map_freq(freq)
industry_name = 'sw'
industry_level = 1


In [4]:
# uqer因子列表
basic_factor_store = factor_store.basic_factor_store
# alpha191因子列表
alpha_factor_store = factor_store.alpha_factor_store


In [5]:
%%time
# 提取Uqer因子
basic_factor_org = engine.fetch_factor_range(universe, basic_factor_store, dates=ref_dates)
# 提取alpha191因子
alpha191_factor_org = engine.fetch_factor_range(universe, alpha_factor_store, dates=ref_dates, used_factor_tables=[Alpha191])


CPU times: user 1min 57s, sys: 2.43 s, total: 1min 59s
Wall time: 2min 16s


In [6]:
%%time
# 合并所有的因子
factor_data_org = pd.merge(basic_factor_org, alpha191_factor_org, on=['trade_date', 'code'], how='outer')


CPU times: user 143 ms, sys: 38 µs, total: 143 ms
Wall time: 142 ms


In [7]:
%%time
# 获取
industry = engine.fetch_industry_range(universe, dates=ref_dates)
factor_data = pd.merge(factor_data_org, industry, on=['trade_date', 'code']).fillna(0.)
risk_total = engine.fetch_risk_model_range(universe, dates=ref_dates)[1]


CPU times: user 1.15 s, sys: 140 ms, total: 1.29 s
Wall time: 2.05 s


In [8]:
%%time
return_data = engine.fetch_dx_return_range(universe, dates=ref_dates, 
                                           horizon=horizon, offset=0,
                                           benchmark = benchmark_code)


CPU times: user 340 ms, sys: 4.04 ms, total: 344 ms
Wall time: 2.26 s


In [9]:
train_data = pd.merge(factor_data_org, return_data, on=['trade_date', 'code']).dropna()
train_data.head()

,trade_date,ACCA,ACD20,ACD6,AD,AD20,AD6,ADTM,ADX,ADXR,...,alpha_93,alpha_94,alpha_95,alpha_96,alpha_97,alpha_98,alpha_99,chgPct_y,secShortName_y,dx
140,2018-01-02,0.355932,0.237288,0.135593,0.457627,0.474576,0.457627,0.050847,0.389831,0.372881,...,3.938425,-45089228.0,3.468182e+07,36.316874,9.957724e+05,-0.496023,-0.027284,0.0255,鱼跃医疗,0.025570
640,2018-01-09,0.355932,0.245763,0.796610,0.474576,0.457627,0.457627,0.186441,0.559322,0.372881,...,2.370992,-4785119.0,4.248298e+07,69.609892,2.521695e+06,-1.378945,-0.818404,0.0314,鱼跃医疗,0.003859
1140,2018-01-16,0.355932,0.694915,0.779661,0.474576,0.457627,0.474576,0.186441,0.661017,0.440678,...,1.587275,4902989.0,4.486341e+07,62.466187,2.372626e+06,-0.992047,-0.747561,-0.0058,鱼跃医疗,-0.008000
1640,2018-01-23,0.355932,0.677966,0.262712,0.491525,0.491525,0.491525,0.389831,0.389831,0.338983,...,2.083298,-8983907.0,4.330308e+07,47.359301,1.570923e+06,-1.001967,-0.536541,-0.0058,鱼跃医疗,0.081827
2140,2018-01-30,0.355932,0.898305,0.949153,0.508475,0.491525,0.491525,0.864407,0.338983,0.508475,...,1.160695,57692591.0,7.459704e+07,67.920513,3.960808e+06,-2.599162,-0.255261,0.0100,鱼跃医疗,0.010549


# 查看因子间相关性

In [10]:
# sns.heatmap(train_data.corr())

In [11]:
%%time
benchmark_total = engine.fetch_benchmark_range(dates=ref_dates, benchmark=benchmark_code)
industry_total = engine.fetch_industry_matrix_range(universe, dates=ref_dates, category=industry_name, level=industry_level)

train_data = pd.merge(factor_data, return_data, on=['trade_date', 'code']).dropna()
train_data.head()

CPU times: user 1.06 s, sys: 484 ms, total: 1.54 s
Wall time: 1.81 s


In [12]:
# Constraintes settings

industry_names = industry_list(industry_name, industry_level)
constraint_risk = ['EARNYILD', 'LIQUIDTY', 'GROWTH', 'SIZE', 'SIZENL', 'BETA', 'MOMENTUM'] + industry_names
# constraint_risk = ['EARNYILD', 'LIQUIDTY', 'GROWTH', 'SIZE', 'BETA', 'MOMENTUM'] + industry_names

total_risk_names = constraint_risk + ['benchmark', 'total']

b_type = []
l_val = []
u_val = []

for name in total_risk_names:
    if name == 'benchmark':
        b_type.append(BoundaryType.RELATIVE)
        l_val.append(0.0)
        u_val.append(1.0)
    elif name == 'total':
        b_type.append(BoundaryType.ABSOLUTE)
        l_val.append(-0.0)
        u_val.append(0.0)
    elif name == 'SIZE':
        b_type.append(BoundaryType.ABSOLUTE)
        l_val.append(-0.1)
        u_val.append(0.1)
    elif name == 'SIZENL':
        b_type.append(BoundaryType.ABSOLUTE)
        l_val.append(-0.1)
        u_val.append(-0.1)
    elif name in industry_names:
        b_type.append(BoundaryType.ABSOLUTE)
        l_val.append(-0.005)
        u_val.append(0.005)
    else:
        b_type.append(BoundaryType.ABSOLUTE)
        l_val.append(-1.0)
        u_val.append(1.0)
# for name in total_risk_names:
#     if name == 'benchmark':
#         b_type.append(BoundaryType.RELATIVE)
#         l_val.append(0.0)
#         u_val.append(1.0)
#     elif name == 'total':
#         b_type.append(BoundaryType.ABSOLUTE)
#         l_val.append(.0)
#         u_val.append(.0)
#     else:
#         b_type.append(BoundaryType.ABSOLUTE)
#         l_val.append(-1.005)
#         u_val.append(1.005)

bounds = create_box_bounds(total_risk_names, b_type, l_val, u_val)


In [13]:
# 获取特征名
features = list(basic_factor_store.keys())
alpha_features = list(alpha_factor_store.keys())
features.extend(alpha_features)

label = ['dx']

In [14]:
%%time
from datetime import datetime, timedelta
from models.m1_xgb import *
from conf.configuration import xgb_conf
import xgboost as xgb
import gc

def create_scenario():
    weight_gap = 1
    transact_cost = 0.003

    executor = NaiveExecutor()
    leverags = []
    trade_dates = []
    current_pos = pd.DataFrame()
    previous_pos = pd.DataFrame()
    tune_record = pd.DataFrame()  
    rets = []
    net_rets = []
    turn_overs = []
    leverags = []
    ics = []

    # take ref_dates[i] as an example
    for i, ref_date in enumerate(back_ref_dates):
        alpha_logger.info('{0} is start'.format(ref_date))

        # machine learning model
        # Filter Training data
        # train data
        trade_date_pre = ref_date - timedelta(days=1)
        trade_date_pre_80 = ref_date - timedelta(days=80)
        
        # train = train_data[(train_data.trade_date <= trade_date_pre) & (trade_date_pre_80 <= train_data.trade_date)].dropna()
        # 训练集构造, 选择当天之前(不含当天)的因子数据作为训练集.
        train = train_data[train_data.trade_date <= trade_date_pre].dropna()

        if len(train) <= 0:
            continue
        x_train = train[features]
        y_train = train[label]
        alpha_logger.info('len_x_train: {0}, len_y_train: {1}'.format(len(x_train.values), len(y_train.values)))
        alpha_logger.info('X_train.shape={0}, X_test.shape = {1}'.format(np.shape(x_train), np.shape(y_train)))

        # xgb_configuration
        xgb_conf.xgb_config_r()
        xgb_conf.cv_folds = None
        xgb_conf.early_stop_round = 10
        xgb_conf.max_round = 80
        tic = time.time()
        # training
        xgb_model = XGBooster(xgb_conf)
        GPU_device = False

        if GPU_device:
            xgb_model.set_params(tree_method='gpu_hist', max_depth=5)
        else:
            xgb_model.set_params(max_depth=5)
        alpha_logger.info('train params {}s'.format(xgb_model.get_params))
        
        best_score, best_round, best_model = xgb_model.fit(x_train, y_train)
        alpha_logger.info('Training time cost {}s'.format(time.time() - tic))
        alpha_logger.info('best_score = {}, best_round = {}'.format(best_score, best_round))
    
        # 测试集, 取当天的因子数据作为输入.
        total_data_test_excess = train_data[train_data.trade_date == ref_date]
        alpha_logger.info('{0} total_data_test_excess: {1}'.format(ref_date, len(total_data_test_excess)))

        if len(total_data_test_excess) <= 0:
            alpha_logger.info('{0} HAS NO DATA!!!'.format(ref_date))
            continue
        
        # 获取当天的行业, 风险模型和基准数据
        industry_matrix = industry_total[industry_total.trade_date == ref_date]
        benchmark_w = benchmark_total[benchmark_total.trade_date == ref_date]
        risk_matrix = risk_total[risk_total.trade_date == ref_date]

        total_data = pd.merge(industry_matrix, benchmark_w, on=['code'], how='left').fillna(0.)
        total_data = pd.merge(total_data, risk_matrix, on=['code'])
        alpha_logger.info('{0} len_of_total_data: {1}'.format(ref_date, len(total_data)))

        total_data_test_excess = pd.merge(total_data, total_data_test_excess, on=['code'])
        alpha_logger.info('{0} len_of_total_data_test_excess: {1}'.format(ref_date, len(total_data_test_excess)))
        
        codes = total_data_test_excess.code.values.tolist()
        alpha_logger.info('{0} full re-balance: {1}'.format(ref_date, len(codes)))
        ## 获取调仓日当天的股票收益
        dx_returns = return_data[return_data.trade_date == ref_date][['code', 'dx']]

        benchmark_w = total_data_test_excess.weight.values
        alpha_logger.info('shape_of_benchmark_w: {}'.format(np.shape(benchmark_w)))
        is_in_benchmark = (benchmark_w > 0.).astype(float).reshape((-1, 1))
        total_risk_exp = np.concatenate([total_data_test_excess[constraint_risk].values.astype(float),
                                         is_in_benchmark,
                                         np.ones_like(is_in_benchmark)],
                                         axis=1)
        alpha_logger.info('shape_of_total_risk_exp_pre: {}'.format(np.shape(total_risk_exp)))
        total_risk_exp = pd.DataFrame(total_risk_exp, columns=total_risk_names)
        alpha_logger.info('shape_of_total_risk_exp: {}'.format(np.shape(total_risk_exp)))
        constraints = LinearConstraints(bounds, total_risk_exp, benchmark_w)
        alpha_logger.info('constraints: {0} in {1}'.format(np.shape(constraints.risk_targets()), ref_date))

        lbound = np.maximum(0., benchmark_w - weight_gap)
        ubound = weight_gap + benchmark_w
        alpha_logger.info('lbound: {0} in {1}'.format(np.shape(lbound), ref_date))
        alpha_logger.info('ubound: {0} in {1}'.format(np.shape(ubound), ref_date))

        # predict
        x_pred = total_data_test_excess[features]
        predict_xgboost = xgb_model.predict(best_model, x_pred)
        a = np.shape(predict_xgboost)
        predict_xgboost = np.reshape(predict_xgboost, (a[0], -1)).astype(np.float64)
        alpha_logger.info('shape_of_predict_xgboost: {}'.format(np.shape(predict_xgboost)))
        del xgb_model
        del best_model
        gc.collect()
        
        # 股票过滤, 组合优化之前过滤掉
        
        # backtest
        try:
            target_pos, _ = er_portfolio_analysis(predict_xgboost,
                                                  total_data_test_excess['industry'].values,
                                                  None,
                                                  constraints,
                                                  False,
                                                  benchmark_w,
                                                  method = 'risk_neutral',
                                                  lbound=lbound,
                                                  ubound=ubound,
                                                  turn_over_target=0.5,
                                                  current_pos=previous_pos)
        except:
            import pdb
            pdb.set_trace()
            alpha_logger.info('target_pos: {}'.format(target_pos))
        alpha_logger.info('target_pos_shape: {}'.format(np.shape(target_pos)))
        alpha_logger.info('len_codes:{}'.format(np.shape(codes)))
        target_pos['code'] = codes
        
        result = pd.merge(target_pos, dx_returns, on=['code'])
        result['trade_date'] = ref_date
        tune_record = tune_record.append(result)
        alpha_logger.info('len_result: {}'.format(len(result)))

        # excess_return = np.exp(result.dx.values) - 1. - index_return.loc[ref_date, 'dx']
        excess_return = np.exp(result.dx.values) - 1.
        ret = result.weight.values @ excess_return
        
        trade_dates.append(ref_date)
        rets.append(np.log(1. + ret))
        alpha_logger.info('len_rets: {}, len_trade_dates: {}'.format(len(rets), len(trade_dates)))
        
        turn_over_org, current_pos = executor.execute(target_pos=target_pos)
        turn_over = turn_over_org / sum(target_pos.weight.values)
        alpha_logger.info('turn_over: {}'.format(turn_over))
        turn_overs.append(turn_over)
        alpha_logger.info('turn_over: {}'.format(turn_over))
        previous_pos = executor.set_current(current_pos)
        alpha_logger.info('current_pos: {}, len_current_pos: {}'.format(previous_pos, len(previous_pos)))
        net_rets.append(np.log(1. + ret - transact_cost * turn_over))        
        alpha_logger.info('len_net_rets: {}, len_trade_dates: {}'.format(len(net_rets), len(trade_dates)))
        alpha_logger.info('{} is finished'.format(ref_date))
    # ret_df = pd.DataFrame({'xgb_regress': rets}, index=trade_dates)
    ret_df = pd.DataFrame({'xgb_regress': rets, 'net_xgb_regress':net_rets}, index=trade_dates)
    ret_df.loc[advanceDateByCalendar('china.sse', ref_dates[-1], freq).strftime('%Y-%m-%d')] = 0.
    ret_df = ret_df.shift(1)
    ret_df.iloc[0] = 0.
    return ret_df, tune_record, rets, net_rets


CPU times: user 3.44 ms, sys: 1.01 ms, total: 4.46 ms
Wall time: 3.52 ms


In [15]:
# 滚动回测
ret_df, tune_record, rets, net_rets = create_scenario()


2019-12-18 00:24:25,270 - ALPHA_MIND - INFO - 2018-10-08 00:00:00 is start
2019-12-18 00:24:25,486 - ALPHA_MIND - INFO - len_x_train: 18500, len_y_train: 18500
2019-12-18 00:24:25,487 - ALPHA_MIND - INFO - X_train.shape=(18500, 614), X_test.shape = (18500, 1)
2019-12-18 00:24:25,487 - ALPHA_MIND - INFO - train params <bound method XGBooster.get_params of <models.m1_xgb.XGBooster object at 0x7f31af80f550>>s
2019-12-18 00:24:25,488 - ../../../models/m1_xgb.py[line:95] - INFO: NonCrossValidation。。。。


[0]	train-rmse:0.498181	train-logloss:0.683149
Multiple eval metrics have been passed: 'train-logloss' will be used for early stopping.

Will train until train-logloss hasn't improved in 10 rounds.
[5]	train-rmse:0.473962	train-logloss:0.635995
[10]	train-rmse:0.450948	train-logloss:0.593129
[15]	train-rmse:0.429079	train-logloss:0.554003
[20]	train-rmse:0.408284	train-logloss:0.518138
[25]	train-rmse:0.388517	train-logloss:0.485167
[30]	train-rmse:0.369722	train-logloss:0.454768
[35]	train-rmse:0.351876	train-logloss:0.426705
[40]	train-rmse:0.33492	train-logloss:0.400721
[45]	train-rmse:0.318803	train-logloss:0.376604
[50]	train-rmse:0.303483	train-logloss:0.354175
[55]	train-rmse:0.28893	train-logloss:0.333291
[60]	train-rmse:0.275112	train-logloss:0.313825
[65]	train-rmse:0.261987	train-logloss:0.295644
[70]	train-rmse:0.249527	train-logloss:0.278646
[75]	train-rmse:0.237691	train-logloss:0.262727
[79]	train-rmse:0.228652	train-logloss:0.250698


2019-12-18 00:24:31,075 - ALPHA_MIND - INFO - Training time cost 5.587636470794678s
2019-12-18 00:24:31,075 - ALPHA_MIND - INFO - best_score = {'best_score': 0.250698}, best_round = 79
2019-12-18 00:24:31,077 - ALPHA_MIND - INFO - 2018-10-08 00:00:00 total_data_test_excess: 0
2019-12-18 00:24:31,078 - ALPHA_MIND - INFO - 2018-10-08 00:00:00 HAS NO DATA!!!
2019-12-18 00:24:31,078 - ALPHA_MIND - INFO - 2018-10-15 00:00:00 is start
2019-12-18 00:24:31,372 - ALPHA_MIND - INFO - len_x_train: 19000, len_y_train: 19000
2019-12-18 00:24:31,372 - ALPHA_MIND - INFO - X_train.shape=(19000, 614), X_test.shape = (19000, 1)
2019-12-18 00:24:31,373 - ALPHA_MIND - INFO - train params <bound method XGBooster.get_params of <models.m1_xgb.XGBooster object at 0x7f31af80f080>>s
2019-12-18 00:24:31,373 - ../../../models/m1_xgb.py[line:95] - INFO: NonCrossValidation。。。。


[0]	train-rmse:0.498424	train-logloss:0.683139
Multiple eval metrics have been passed: 'train-logloss' will be used for early stopping.

Will train until train-logloss hasn't improved in 10 rounds.
[5]	train-rmse:0.474212	train-logloss:0.635974
[10]	train-rmse:0.451188	train-logloss:0.593068
[15]	train-rmse:0.429304	train-logloss:0.553892
[20]	train-rmse:0.408507	train-logloss:0.518004
[25]	train-rmse:0.388739	train-logloss:0.485012
[30]	train-rmse:0.369958	train-logloss:0.454614
[35]	train-rmse:0.352098	train-logloss:0.42651
[40]	train-rmse:0.335138	train-logloss:0.4005
[45]	train-rmse:0.319028	train-logloss:0.376374
[50]	train-rmse:0.303712	train-logloss:0.353931
[55]	train-rmse:0.289165	train-logloss:0.333035
[60]	train-rmse:0.275347	train-logloss:0.313548
[65]	train-rmse:0.262226	train-logloss:0.295352
[70]	train-rmse:0.249762	train-logloss:0.278326
[75]	train-rmse:0.237931	train-logloss:0.262385
[79]	train-rmse:0.228896	train-logloss:0.250336


2019-12-18 00:24:37,333 - ALPHA_MIND - INFO - Training time cost 5.9601521492004395s
2019-12-18 00:24:37,334 - ALPHA_MIND - INFO - best_score = {'best_score': 0.250336}, best_round = 79
2019-12-18 00:24:37,335 - ALPHA_MIND - INFO - 2018-10-15 00:00:00 total_data_test_excess: 0
2019-12-18 00:24:37,336 - ALPHA_MIND - INFO - 2018-10-15 00:00:00 HAS NO DATA!!!
2019-12-18 00:24:37,336 - ALPHA_MIND - INFO - 2018-10-22 00:00:00 is start
2019-12-18 00:24:37,645 - ALPHA_MIND - INFO - len_x_train: 19500, len_y_train: 19500
2019-12-18 00:24:37,645 - ALPHA_MIND - INFO - X_train.shape=(19500, 614), X_test.shape = (19500, 1)
2019-12-18 00:24:37,646 - ALPHA_MIND - INFO - train params <bound method XGBooster.get_params of <models.m1_xgb.XGBooster object at 0x7f31af80feb8>>s
2019-12-18 00:24:37,646 - ../../../models/m1_xgb.py[line:95] - INFO: NonCrossValidation。。。。


...

2019-12-18 00:33:41,051 - ALPHA_MIND - INFO - Training time cost 17.07945418357849s
2019-12-18 00:33:41,051 - ALPHA_MIND - INFO - best_score = {'best_score': 0.251164}, best_round = 79
2019-12-18 00:33:41,054 - ALPHA_MIND - INFO - 2019-09-09 00:00:00 total_data_test_excess: 0
2019-12-18 00:33:41,054 - ALPHA_MIND - INFO - 2019-09-09 00:00:00 HAS NO DATA!!!
2019-12-18 00:33:41,055 - ALPHA_MIND - INFO - 2019-09-17 00:00:00 is start
2019-12-18 00:33:41,947 - ALPHA_MIND - INFO - len_x_train: 41995, len_y_train: 41995
2019-12-18 00:33:41,948 - ALPHA_MIND - INFO - X_train.shape=(41995, 614), X_test.shape = (41995, 1)
2019-12-18 00:33:41,949 - ALPHA_MIND - INFO - train params <bound method XGBooster.get_params of <models.m1_xgb.XGBooster object at 0x7f31dc0e4b70>>s
2019-12-18 00:33:41,949 - ../../../models/m1_xgb.py[line:95] - INFO: NonCrossValidation。。。。


[0]	train-rmse:0.498233	train-logloss:0.683153
Multiple eval metrics have been passed: 'train-logloss' will be used for early stopping.

Will train until train-logloss hasn't improved in 10 rounds.
[5]	train-rmse:0.474047	train-logloss:0.636057
[10]	train-rmse:0.451055	train-logloss:0.593225
[15]	train-rmse:0.429196	train-logloss:0.554105
[20]	train-rmse:0.40842	train-logloss:0.518264
[25]	train-rmse:0.388677	train-logloss:0.485325
[30]	train-rmse:0.369917	train-logloss:0.454969
[35]	train-rmse:0.352091	train-logloss:0.426925
[40]	train-rmse:0.335154	train-logloss:0.400957
[45]	train-rmse:0.319055	train-logloss:0.376854
[50]	train-rmse:0.303764	train-logloss:0.354454
[55]	train-rmse:0.289237	train-logloss:0.333594
[60]	train-rmse:0.275447	train-logloss:0.314154
[65]	train-rmse:0.262347	train-logloss:0.295995
[70]	train-rmse:0.249907	train-logloss:0.279013
[75]	train-rmse:0.238096	train-logloss:0.26311
[79]	train-rmse:0.229087	train-logloss:0.251116


2019-12-18 00:33:58,684 - ALPHA_MIND - INFO - Training time cost 16.735140085220337s
2019-12-18 00:33:58,684 - ALPHA_MIND - INFO - best_score = {'best_score': 0.251116}, best_round = 79
2019-12-18 00:33:58,687 - ALPHA_MIND - INFO - 2019-09-17 00:00:00 total_data_test_excess: 0
2019-12-18 00:33:58,688 - ALPHA_MIND - INFO - 2019-09-17 00:00:00 HAS NO DATA!!!
2019-12-18 00:33:58,688 - ALPHA_MIND - INFO - 2019-09-24 00:00:00 is start
2019-12-18 00:33:59,427 - ALPHA_MIND - INFO - len_x_train: 42495, len_y_train: 42495
2019-12-18 00:33:59,427 - ALPHA_MIND - INFO - X_train.shape=(42495, 614), X_test.shape = (42495, 1)
2019-12-18 00:33:59,428 - ALPHA_MIND - INFO - train params <bound method XGBooster.get_params of <models.m1_xgb.XGBooster object at 0x7f31af80f550>>s
2019-12-18 00:33:59,428 - ../../../models/m1_xgb.py[line:95] - INFO: NonCrossValidation。。。。


[0]	train-rmse:0.498232	train-logloss:0.683156
Multiple eval metrics have been passed: 'train-logloss' will be used for early stopping.

Will train until train-logloss hasn't improved in 10 rounds.
[5]	train-rmse:0.47404	train-logloss:0.636049
[10]	train-rmse:0.451041	train-logloss:0.593202
[15]	train-rmse:0.429181	train-logloss:0.554084
[20]	train-rmse:0.408413	train-logloss:0.518256
[25]	train-rmse:0.388667	train-logloss:0.485311
[30]	train-rmse:0.3699	train-logloss:0.454947
[35]	train-rmse:0.352074	train-logloss:0.426903
[40]	train-rmse:0.335136	train-logloss:0.400937
[45]	train-rmse:0.319038	train-logloss:0.376835
[50]	train-rmse:0.303746	train-logloss:0.354434
[55]	train-rmse:0.289226	train-logloss:0.333587
[60]	train-rmse:0.275434	train-logloss:0.314145
[65]	train-rmse:0.262336	train-logloss:0.29599
[70]	train-rmse:0.249896	train-logloss:0.279011
[75]	train-rmse:0.238085	train-logloss:0.26311
[79]	train-rmse:0.229072	train-logloss:0.251117


2019-12-18 00:34:16,836 - ALPHA_MIND - INFO - Training time cost 17.407888174057007s
2019-12-18 00:34:16,837 - ALPHA_MIND - INFO - best_score = {'best_score': 0.251117}, best_round = 79
2019-12-18 00:34:16,839 - ALPHA_MIND - INFO - 2019-09-24 00:00:00 total_data_test_excess: 0
2019-12-18 00:34:16,840 - ALPHA_MIND - INFO - 2019-09-24 00:00:00 HAS NO DATA!!!


In [16]:
tune_record.head(500)

""


In [17]:
from pyecharts import options as opts
from example.commons import Collector
from pyecharts.charts import Line

plot = ret_df[['xgb_regress', 'net_xgb_regress']].cumsum()
v1 = list(plot.index)
v2 = list(plot.xgb_regress)
v3 = list(plot.net_xgb_regress)

line_chart = (
    Line()
    .add_xaxis(v1)
    .add_yaxis("xgb_regress", v2)
    .add_yaxis("net_xgb_regress", v3)
    .set_series_opts(
        label_opts=opts.LabelOpts(is_show=False),
    )
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(is_scale=True),
        yaxis_opts=opts.AxisOpts(
            is_scale=True,
            splitarea_opts=opts.SplitAreaOpts(
                is_show=True, areastyle_opts=opts.AreaStyleOpts(opacity=1)
            ),
        ),
        datazoom_opts=[opts.DataZoomOpts(pos_bottom="-1%")],
        title_opts=opts.TitleOpts(title='Fixed freq rebalanced: {0}'.format(freq)),
    )
)

line_chart.render_notebook()

In [18]:
# # 调仓记录保存
from datetime import datetime
import pandas as pd
import sqlite3
con = sqlite3.connect('./tune_record.db')
# tune_record.to_sql('tune_record', con=con, if_exists='append', index=False)
sql = 'select * from tune_record'
tune_record = pd.read_sql(sql, con)
tune_record[['trade_date','code','portfolio_weight']].head()

DatabaseError: Execution failed on sql 'select * from tune_record': no such table: tune_record

In [ ]:
 ##--------------因子组合回测--------------
import json
import pdb
from rqalpha.api import *
from rqalpha import run_func
class DailyDefaultStrategy(object):
    def __init__(self, session, stock_sets_ob):
        self._stock_sets_ob = json.loads(stock_sets_ob)
        self._session = session
        self._start_time_str = datetime.now().strftime('%m-%d %H:%M')
    
    
    def init(self,context):
        #读取股票池文件
        context.stock_sets = pd.DataFrame(self._stock_sets_ob)
        context.trade_date_list= list(set(context.stock_sets.trade_date))
        context.holding_stock_df = None
        
        # 是否已发送了order
        context.fired = False

    #盘前处理
    def before_trading(self, context, bar_dict):
        context.trade_signal = False
        date = context.now.date()
        if str(date) in context.trade_date_list:
            context.trade_signal = True

    #盘后处理
    def after_trading(self, context):
        pass
    
    # 你选择的证券的数据更新将会触发此段逻辑，例如日或分钟历史数据切片或者是实时数据切片更新
    def handle_bar(self, context, bar_dict):
        # 开始编写你的主要的算法逻辑
        if context.trade_signal == False:
            return
        date = context.now.date()
        # stock_dict = context.stock_sets.set_index('trade_date').loc[str(date)]
        stock_set = context.stock_sets
        stock_dict = stock_set[stock_set['trade_date'] == str(date)].set_index('trade_date')
        stock_df = self.filter_specials(stock_dict, context)
        #剔除后的股票
        if 'portfolio_weight' in list(set(stock_df.columns)):
            self.rebalance_weight(context, stock_df)
        else:
            self.rebalance_equal(context, stock_df)
   
    def _industry_distribute(self, stock_positions, industries):
        industry_sets_dict = {}
        industries = industries.rename(columns={'symbol':'code'})
        stock_positions = stock_positions.reset_index().rename(
            columns={'symbol':'name','order_book_id':'code','date':'trade_date'})
        stock_positions['trade_date'] = stock_positions['trade_date'].apply(lambda x: x.strftime('%Y-%m-%d'))
        industries['trade_date'] = industries['trade_date'].apply(lambda x: x.strftime('%Y-%m-%d'))
        stock_positions = stock_positions.merge(industries, on=['trade_date','code'])
        industry_grouped = stock_positions.dropna().groupby(['trade_date'])
        for k, g in industry_grouped:
            gt = g.groupby(['industry_code','industry']).count().reset_index()[['industry_code','industry','code']]
            gt['ratio'] = gt['code']/gt['code'].values.sum()
            gt=gt.rename(columns={'industry':'industry_name'})
            industry_sets_dict[k] = gt[['industry_code','industry_name','ratio']].to_dict(orient='records')
        return industry_sets_dict
    
    def _month_profit(self, portfolio, stock_capital):
        pd.DataFrame(portfolio.reset_index()).shift(1)
        month_profit = portfolio['total_value'].resample('M').mean()
        month_profit = pd.DataFrame(month_profit.reset_index())
        month_profit['last_total'] = month_profit['total_value'].shift(1)
        month_profit = month_profit.fillna(stock_capital)
        month_profit['month_ratio'] = np.log(month_profit.total_value / month_profit.last_total)
        return month_profit.rename(columns={'date':'trade_date'})
    
    def set_result(self, result):
        #summary 结算信息
        summary = result['sys_analyser']['summary']
        #trades 交易记录
        trades = result['sys_analyser']['trades']
        #portfolio 收益曲线
        portfolio = result['sys_analyser']['portfolio']
        # benchmark_portfolio 基础收益曲线benchmark_portfolio
        benchmark_portfolio = result['sys_analyser']['benchmark_portfolio']
        # stock_account 个人收益
        stock_account = result['sys_analyser']['stock_account']
        # stock_positions 股票持仓情况
        stock_positions = result['sys_analyser']['stock_positions']
        
        
    def filter_specials(self, stock_dict, context):
        stock_df = pd.DataFrame(stock_dict)
        stock_list = list(stock_df.code)
        stock_list=[stock for stock in stock_list]
        
        return stock_df.set_index('code').loc[stock_list,:] 
    
    def rebalance_equal(self, context, stock_df):
        holding_list = list(set(stock_df.index))
        if len(holding_list) > 0:
            every_stock = context.portfolio.portfolio_value/len(holding_list) 
        # 空仓只有买入操作
        if len(list(context.portfolio.positions.keys()))==0:
            for stock_to_buy in list(holding_list):
                #print(stock_to_buy)
                order_target_value(stock_to_buy,every_stock)
        else:
            for stock_to_sell in list(context.portfolio.positions.keys()):
                if stock_to_sell not in list(holding_list):
                    order_target_value(stock_to_sell, 0)
            for stock_to_buy in list(holding_list):
                order_target_value(stock_to_buy, every_stock)
                
        
    def rebalance_weight(self, context, stock_df):
        #没有持仓则全部下单
        if len(list(context.portfolio.positions.keys())) == 0:
            weight_sum = stock_df.portfolio_weight.sum()
            every_values = context.portfolio.total_value / weight_sum
            for index, weight in stock_df.iterrows():
                order_target_value(index, weight.portfolio_weight * every_values)
            context.holding_stock_df = stock_df
        else:
            now_df = context.holding_stock_df #当前持有的股票
            # 做交集查看上次实际成交股票
            intersect = list(set(now_df.index).intersection(set(context.portfolio.stock_account.positions.keys())))
            if len(intersect) <= 0:
                now_df = pd.DataFrame({'now_weight':[]})
            else:
                now_df = now_df.loc[context.portfolio.stock_account.positions.keys()]
                now_df = now_df.rename(columns={'portfolio_weight':'now_weight'}) #持仓
            # 做并集
            stock_df = stock_df.rename(columns={'portfolio_weight':'next_weight'}) #调仓
            now_df = pd.merge(now_df, stock_df, left_index=True, right_index=True, how='outer')
            now_df = now_df.fillna(0)
            now_df['diff_weight'] = now_df.next_weight - now_df.now_weight #调仓股票和持仓股票进行对比
            buy_stock = now_df[now_df['diff_weight'] > 0] #当前需买进
            sell_stock = now_df[now_df['diff_weight'] < 0]
            
            # 根据当前权重分配，目的是持仓达到和调仓的权重一样
            weight_sum = stock_df.next_weight.sum()
            every_values = context.portfolio.total_value/ weight_sum
            
            # 平掉不需要股票
            for index, weight in sell_stock.iterrows():
                if weight.now_weight == abs(weight.diff_weight): #全部清仓
                    order_target_value(index, 0)
                else:# 调整权重，此处会有误差，每期的every_values不一样
                    order_target_value(index, weight.diff_weight * every_values)
            # 新建增仓
            for index, weight in buy_stock.iterrows():
                order_target_value(index, abs(weight.diff_weight) * every_values)
            stock_df = stock_df.rename(columns={'next_weight':'portfolio_weight'})
            context.holding_stock_df = stock_df

In [ ]:
def ricequant_backtesting(conmbine):
    benchmark_info = '000905.XSHG'
    conmbine = conmbine.copy()
    conmbine['trade_date'] = conmbine['trade_date'].apply(lambda x: x.split(' ')[0] if type(x) is str else x.date().strftime("%Y-%m-%d"))
    #stk_result = stk_data[stk_data['portfolio_weight']>0.001]#.set_index('trade_date').loc['2018-04-12']
    conmbine['code'] = conmbine['code'].apply(lambda x: "{:06d}".format(x) + '.XSHG' 
                                              if len(str(x))==6 and str(x)[0] in '6' 
                                              else "{:06d}".format(x)+ '.XSHE')
    backtesting = DailyDefaultStrategy(1234412, json.dumps(conmbine.to_dict(orient='records')))
    trade_date_list = list(set(conmbine.trade_date))
    trade_date_list.sort(reverse=False)
    base_config = {}
    base_config['start_date'] = trade_date_list[0]
    base_config['end_date'] = trade_date_list[-1]
    base_config['benchmark'] = benchmark_info
            
    accounts_config = {}
    accounts_config['stock'] = 100000000
    base_config['accounts'] = accounts_config
    base_config['frequency'] = '1d'
    base_config['skip_suspended'] = True
    
    extra_config = {}

    mod_config = {}
    sys_analyser_config = {}
    sys_analyser_config['enabled'] = True
    sys_analyser_config['plot'] = True
    mod_config['sys_analyser'] = sys_analyser_config
        
    config = {}
    config['base'] = base_config
    config['extra'] = extra_config
    config['mod'] = mod_config
    result = run_func(init=backtesting.init, before_trading=backtesting.before_trading, 
                  after_trading=backtesting.after_trading, handle_bar=backtesting.handle_bar, config=config)
    return result

In [ ]:
%matplotlib inline
tune_record  = tune_record.rename(columns={'weight':'portfolio_weight'})
#处理权重小, 直接处理为0
# tune_record = tune_record[tune_record['portfolio_weight'] >= 0.05]
result = ricequant_backtesting(tune_record[['trade_date','code','portfolio_weight']])

In [ ]:
tune_record[tune_record['trade_date'] == '2019-06-05']

# 回测记录

In [ ]:
print(result.keys())
record = result['sys_analyser']
print(record.keys())
trade_detail = record['trades']
stock_account = record['stock_account']
stock_positions = record['stock_positions']
stock_positions.head()

In [ ]:
# 获取指定交易日期的交易记录
trade_detail[trade_detail['trading_datetime'] == '2018-11-29 15:00:00']

In [ ]:
trade_detail[trade_detail['trading_datetime'] == '2018-12-06 15:00:00']

In [ ]:
# 获取指定交易日的持仓仓位
stock_positions[(stock_positions.index == '2018-12-20') & (stock_positions.quantity != 0)].reset_index()

In [ ]:
stock_positions[(stock_positions.index == '2018-12-27') & (stock_positions.quantity != 0)].reset_index()

In [ ]:
stock_positions[(stock_positions.index == '2019-01-07') & (stock_positions.quantity != 0)].reset_index()

In [ ]:
# trade_detail.to_csv('./trades_record.csv', encoding="utf_8_sig")